In [126]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()

In [127]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [128]:
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [129]:
nombrePersonalizado = "Programa Nutricional"
pacienteId = "0018c00002anTmeAAE"
queryPrograma = "SELECT Id FROM HealthCloudGA__Program__c WHERE Name = '" + nombrePersonalizado + "' AND Paciente__c = '" + pacienteId + "' LIMIT 1" 
vllamadaEmpresa = sf.smart_query(queryPrograma)
#vllamadaEmpresa.head()

100%|██████████| 1/1 [00:03<00:00,  3.05s/it]


In [130]:

queryPrograma = "SELECT Id FROM case WHERE Program__c = '" + vllamadaEmpresa["Id"][0] + "'"
vllamadaEmpresa = sf.smart_query(queryPrograma)
vllamadaEmpresa.head()
cntdCase = len(vllamadaEmpresa.index)
#print (len(vllamadaEmpresa.index))

100%|██████████| 1/1 [00:02<00:00,  2.99s/it]


In [131]:
#idGoal2 = pd.DataFrame()
def traerGoal(iD):
    idGoalDF = pd.DataFrame()
    #idGoalTotal = pd.DataFrame()
    idCase = iD
    queryGoal = "SELECT Id FROM HealthCloudGA__CarePlanGoal__c WHERE Etapa__c = '" + idCase + "'"
    idGoalDF = sf.smart_query(queryGoal)
    #print (len(idGoalDF.index))
    #print (idGoalDF)
    return idGoalDF

In [132]:
idGoal2 = pd.DataFrame()  # Declare idGoal2 as a global variable
derty = pd.DataFrame()

#idGoal2 = vllamadaEmpresa["Id"].apply(lambda y: traerGoal(y))
for registros in range(cntdCase):
    idGoal2= traerGoal(vllamadaEmpresa["Id"][registros])
    derty = pd.concat([derty, idGoal2], axis=0)
cntdGoal = len(derty.index)
#derty.head(10)

100%|██████████| 1/1 [00:02<00:00,  2.52s/it]


In [133]:
def traerActividad(iD):
    idGoalDF = pd.DataFrame()
    #idGoalTotal = pd.DataFrame()
    idCase = iD
    queryGoal = "SELECT Id FROM Actividad_Programa__c WHERE Goal__c = '" + idCase + "'"
    idGoalDF = sf.smart_query(queryGoal)
    #print (len(idGoalDF.index))
    #print (idGoalDF)
    return idGoalDF

In [134]:
derty.reset_index(drop=True, inplace=True)
derty["Id"][0]

'a0HRN0000013AVJ2A2'

In [135]:
dfActividad = pd.DataFrame()  # Declare idGoal2 as a global variable
dertyActividad = pd.DataFrame()

#idGoal2 = vllamadaEmpresa["Id"].apply(lambda y: traerGoal(y))
for registros in range(cntdGoal):
    dfActividad = traerActividad(derty["Id"][registros])
    dertyActividad = pd.concat([dertyActividad, dfActividad], axis=0)
#cntdGoal = len(derty.index)
dertyActividad.head(10)

100%|██████████| 1/1 [00:02<00:00,  2.42s/it]


,Id
0,a43RN0000001EnJYAU
1,a43RN0000001EnKYAU
0,a43RN0000001EnLYAU
1,a43RN0000001EnMYAU
2,a43RN0000001EnNYAU
3,a43RN0000001EnOYAU
4,a43RN0000001EnPYAU
5,a43RN0000001EnQYAU
6,a43RN0000001EnRYAU
7,a43RN0000001EnSYAU


In [136]:
dertyActividad.reset_index(drop=True, inplace=True)
dertyActividad["Id"][0]
dertyActividad.head()

,Id
0,a43RN0000001EnJYAU
1,a43RN0000001EnKYAU
2,a43RN0000001EnLYAU
3,a43RN0000001EnMYAU
4,a43RN0000001EnNYAU
